In [2]:
import argparse
import torch
import sysbinder
import os
import torch.nn.functional as F
import torch.multiprocessing as mp
import scipy.optimize
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from rtpt import RTPT
from tqdm import tqdm
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from data import CLEVREasy_1_WithAnnotations, CLEVR4_1_WithAnnotations
from retrievalbinder import NeuralConceptBinder
import utils_bnr as utils_bnr

torch.set_num_threads(40)
OMP_NUM_THREADS = 40
MKL_NUM_THREADS = 40

class Args:
    seed = 0
    batch_size = 30
    num_workers = 4
    image_size = 128
    image_channels = 3
    
    # !!!! CHANGE THIS AS REQUIRED !!!!
#     base_save_path = '/workspace/repositories/SysBindRetrieve/logs/clevr_easy_500_epochs/random_imgs'
#     data_path = "/workspace/datasets_wolf/CLEVR-Easy-1/"

    base_save_path = '/workspace/repositories/SysBindRetrieve/logs/clevr4_600_epochs/random_imgs'
    data_path = "/workspace/datasets_wolf/CLEVR-4-1/"
    
args=Args()
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
utils_bnr.set_seed(args.seed)

# set seed
torch.manual_seed(args.seed)


# we train the classifier on the original validation set and test on the original test set
if 'CLEVR-Easy-1' in args.data_path:
    train_dataset = CLEVREasy_1_WithAnnotations(
        root=args.data_path, phase="train", img_size=args.image_size, max_num_objs=4,
        num_categories=2, perc_imgs=1.
    )
elif 'CLEVR-4-1' in args.data_path:
    train_dataset = CLEVR4_1_WithAnnotations(
        root=args.data_path, phase="train", img_size=args.image_size, max_num_objs=4,
        num_categories=4, perc_imgs=1.
    )

loader_kwargs = {
    "batch_size": args.batch_size,
    "shuffle": False,
    "num_workers": args.num_workers,
    "pin_memory": True,
    "drop_last": True,
}
train_loader = DataLoader(train_dataset, **loader_kwargs)

ImportError: cannot import name 'CLEVREasy_1_WithAnnotations' from 'data' (/workspace/repositories/SysBindRetrieve/sysbinder/data.py)

In [ ]:
for i, batch in enumerate(train_loader):
    imgs, _, _, _ = batch
    np_imgs = np.transpose(imgs.detach().cpu().numpy(), (0, 2, 3, 1))[:25]

    fig, axs = plt.subplots(5, 5)
    axs = axs.flatten()
    for j in range(len(axs)):
        if j < len(np_imgs):
            axs[j].imshow(np_imgs[j])
        axs[j].axis('off')
    # fig.suptitle("Random Imgs")
    # plt.show()
    fig.savefig(f'{args.base_save_path}/random_imgs_{i}.png', bbox_inches='tight')
    if i == 5:
        break